In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.layers import Dense, Dropout, LSTM, Bidirectional, Input, TimeDistributed, Attention, GlobalAveragePooling2D
from tensorflow.keras.layers import LayerNormalization, Add
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
UCF_CRIME_PATH = '...../RAD_dataset/'
LABELS = ['Accident', 'Carfire', 'Fighting', 'Snatching']


In [ ]:
def video_to_frames(video_path, max_frames=25):
    frames = []
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    interval = max(1, frame_count // max_frames)
    for i in range(0, frame_count, interval):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (224, 224))
        frames.append(frame)
        if len(frames) >= max_frames:
            break
    cap.release()
    return np.array(frames)

In [ ]:
class VideoDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, video_paths, labels, batch_size, max_frames, num_classes):
        self.video_paths = video_paths
        self.labels = labels
        self.batch_size = batch_size
        self.max_frames = max_frames
        self.num_classes = num_classes

    def __len__(self):
        return len(self.video_paths) // self.batch_size

    def __getitem__(self, idx):
        batch_paths = self.video_paths[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_labels = self.labels[idx * self.batch_size:(idx + 1) * self.batch_size]

        batch_frames = [video_to_frames(path, self.max_frames) for path in batch_paths]
        batch_frames = np.array(batch_frames)
        batch_labels = to_categorical(batch_labels, self.num_classes)

        return batch_frames, batch_labels

In [ ]:
def create_residual_bilstm_block(input_tensor, lstm_units):
    x = Bidirectional(LSTM(lstm_units, return_sequences=True))(input_tensor)
    attention_output = Attention()([x, x])
    x = Add()([x, attention_output])  # Residual connection
    x = LayerNormalization()(x)
    return x

def create_densenet_bilstm_attention_model(num_classes, lstm_units=128, num_blocks=3):
    input_tensor = Input(shape=(None, 224, 224, 3))
    base_model = DenseNet201(include_top=False, input_shape=(224, 224, 3), weights='imagenet')
    base_model.trainable = False

    x = TimeDistributed(base_model)(input_tensor)
    x = TimeDistributed(GlobalAveragePooling2D())(x)

    for _ in range(num_blocks):
        x = create_residual_bilstm_block(x, lstm_units)

    x = LSTM(lstm_units)(x)
    output_tensor = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=input_tensor, outputs=output_tensor)
    return model

In [ ]:
num_classes = len(LABELS)
lstm_units = 128
num_blocks = 3

In [ ]:
model = create_densenet_bilstm_attention_model(num_classes=num_classes, lstm_units=lstm_units, num_blocks=num_blocks)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
video_paths = [os.path.join(UCF_CRIME_PATH, label, video) for label in LABELS for video in os.listdir(os.path.join(UCF_CRIME_PATH, label))]
labels = [LABELS.index(label) for label in LABELS for _ in os.listdir(os.path.join(UCF_CRIME_PATH, label))]

In [ ]:
split_index = int(0.8 * len(video_paths))
train_paths, val_paths = video_paths[:split_index], video_paths[split_index:]
train_labels, val_labels = labels[:split_index], labels[split_index:]

In [ ]:
train_generator = VideoDataGenerator(train_paths, train_labels, batch_size=8, max_frames=25, num_classes=num_classes)
val_generator = VideoDataGenerator(val_paths, val_labels, batch_size=8, max_frames=25, num_classes=num_classes)

In [ ]:
history = model.fit(train_generator, validation_data=val_generator, epochs=50)
model.save('model.h5')
y_true = val_labels
y_pred_probs = model.predict(val_generator)
y_pred = np.argmax(y_pred_probs, axis=1)


In [ ]:
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=LABELS))

In [ ]:
conf_matrix = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Red", xticklabels=LABELS, yticklabels=LABELS)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
plt.figure(figsize=(10, 8))
for i in range(num_classes):
    fpr, tpr, _ = roc_curve(to_categorical(y_true, num_classes)[:, i], y_pred_probs[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f'{LABELS[i]} (AUC = {roc_auc:.2f})')


In [ ]:
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='best')
plt.show()

In [ ]:
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

In [ ]:
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

In [ ]:
plt.show()